# The unreasonable effectiveness of Character-level Language Models
## (and why RNNs are still cool)

### [Yoav Goldberg](http://www.cs.biu.ac.il/~yogo)

RNNs, LSTMs and Deep Learning are all the rage, and a recent [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Andrej Karpathy is doing a great job explaining what these models are and how to train them.
It also provides some very impressive results of what they are capable of.  This is a great post, and if you are interested in natural language, machine learning or neural networks you should definitely read it. 

Go read it now, then come back here. 

You're back? good. Impressive stuff, huh? How could the network learn to immitate the input like that?
Indeed. I was quite impressed as well.

However, it feels to me that most readers of the post are impressed by the wrong reasons.
This is because they are not familiar with **unsmoothed maximum-liklihood character level language models** and their unreasonable effectiveness at generating rather convincing natural language outputs.

In what follows I will briefly describe these character-level maximum-likelihood langauge models, which are much less magical than RNNs and LSTMs, and show that they too can produce a rather convincing Shakespearean prose. I will also show about 30 lines of python code that take care of both training the model and generating the output. Compared to this baseline, the RNNs may seem somehwat less impressive. So why was I impressed? I will explain this too, below.

## Unsmoothed Maximum Likelihood Character Level Language Model 

The name is quite long, but the idea is very simple.  We want a model whose job is to guess the next character based on the previous $n$ letters. For example, having seen `ello`, the next characer is likely to be either a commma or space (if we assume is is the end of the word "hello"), or the letter `w` if we believe we are in the middle of the word "mellow". Humans are quite good at this, but of course seeing a larger history makes things easier (if we were to see 5 letters instead of 4, the choice between space and `w` would have been much easier).

We will call $n$, the number of letters we need to guess based on, the _order_ of the language model.

RNNs and LSTMs can potentially learn infinite-order language model (they guess the next character based on a "state" which supposedly encode all the previous history). We here will restrict ourselves to a fixed-order language model.

So, we are seeing $n$ letters, and need to guess the $n+1$th one. We are also given a large-ish amount of text (say, all of Shakespear works) that we can use. How would we go about solving this task?

Mathematically, we would like to learn a function $P(c | h)$. Here, $c$ is a character, $h$ is a $n$-letters history, and $P(c|h)$ stands for how likely is it to see $c$ after we've seen $h$.

Perhaps the simplest approach would be to just count and divide (a.k.a **maximum likelihood estimates**). We will count the number of times each letter $c'$ appeared after $h$, and divide by the total numbers of letters appearing after $h$. The **unsmoothed** part means that if we did not see a given letter following $h$, we will just give it a probability of zero.

And that's all there is to it.


### Training Code
Here is the code for training the model. `fname` is a file to read the characters from. `order` is the history size to consult. Note that we pad the data with leading `~` so that we also learn how to start.


In [2]:
from collections import *

def train_char_lm(fname, order=4):
    data = open(fname).read()
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

Let's train it on Andrej's Shakespears's text:

In [3]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt

--2024-10-02 08:57:06--  http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt [following]
--2024-10-02 08:57:09--  https://www.cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt
Resolving www.cs.stanford.edu (www.cs.stanford.edu)... 23.216.154.115, 23.216.154.123
Connecting to www.cs.stanford.edu (www.cs.stanford.edu)|23.216.154.115|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-10-02 08:57:11 ERROR 404: Not Found.



In [4]:
lm = train_char_lm("shakespeare_input.txt", order=4)

Ok. Now let's do some queries:

In [5]:
lm['then']

[('?', 0.045532157085941945),
 (' ', 0.5156516789982926),
 ('c', 0.0443938531587934),
 ('.', 0.035287421741605006),
 (',', 0.21457029026750143),
 (';', 0.02902675014228799),
 (':', 0.017643710870802503),
 ('!', 0.00796812749003984),
 ('\n', 0.02390438247011952),
 ("'", 0.006260671599317018),
 ('s', 0.03414911781445646),
 ('o', 0.0005691519635742744),
 ('i', 0.021627774615822423),
 ('-', 0.0005691519635742744),
 ('t', 0.0022766078542970974),
 ('e', 0.0005691519635742744)]

In [6]:
lm['Firs']

[('t', 1.0)]

In [7]:
lm['rst ']

[('C', 0.09550561797752809),
 ('f', 0.011235955056179775),
 ('i', 0.016853932584269662),
 ('t', 0.05377207062600321),
 ('u', 0.0016051364365971107),
 ('S', 0.16292134831460675),
 ('h', 0.019261637239165328),
 ('s', 0.03290529695024077),
 ('R', 0.0008025682182985554),
 ('b', 0.024879614767255216),
 ('c', 0.012841091492776886),
 ('O', 0.018459069020866775),
 ('w', 0.024077046548956663),
 ('a', 0.02247191011235955),
 ('m', 0.02247191011235955),
 ('n', 0.020064205457463884),
 ('I', 0.009630818619582664),
 ('L', 0.10674157303370786),
 ('M', 0.0593900481540931),
 ('l', 0.01043338683788122),
 ('o', 0.030497592295345103),
 ('H', 0.0040128410914927765),
 ('d', 0.015248796147672551),
 ('W', 0.033707865168539325),
 ('K', 0.008025682182985553),
 ('q', 0.0016051364365971107),
 ('G', 0.0898876404494382),
 ('g', 0.011235955056179775),
 ('k', 0.0040128410914927765),
 ('e', 0.0032102728731942215),
 ('y', 0.002407704654895666),
 ('r', 0.0072231139646869984),
 ('p', 0.00882825040128411),
 ('A', 0.0056179

So `ello` is followed by either space, punctuation or `w` (or `r`, `u`, `n`), `Firs` is pretty much deterministic, and the word following `ist ` can start with pretty much every letter.

### Generating from the model
Generating is also very simple. To generate a letter, we will take the history, look at the last $order$ characteters, and then sample a random letter based on the corresponding distribution.

In [8]:
from random import random

def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

To generate a passage of $k$ characters, we just seed it with the initial history and run letter generation in a loop, updating the history at each turn.

In [9]:
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

### Generated Shakespeare from different order models

Let's try to generate text based on different language-model orders. Let's start with something silly:

### order 2:

In [10]:
lm = train_char_lm("shakespeare_input.txt", order=2)
print(generate_text(lm, 2))

Fifick's se, clesse; leave a knot im of
yes will theithear of Capprinjus mas the
HELING Lordearing!
Ten by I
anchis an her, a gret muc, and stunmad!
HARING Lorthis the dem-se chatherem, wixem,
Whis thentlea, bable sind, be anmaressil.

Cap,
th trauddre.

So hines. Ther mise
Thes; athe my for cond arm suld wed, and ve the'ld setim my peaccappeaf the mor
And al?

Bries any sh it to her'd vill ow ther's See, a thalre ford, in you wer.

Do, hinglass? nork whicas,--

Gen innere. Chat sex orry?

Lor th!

Go of no eave
caugh, of a jestlege.

Yetenceress,
BER:
Ame hime travill.
O'ert ther:
Thumsen no singthenter like I have godgeard:
Weret hentakeed, affishou wild thingen eve mayse doisdo Rome?
Givererear,
SUS:
Them nown a wit lius' this fre fown
ashe cre neve chou pler go IV:
Comme ke ENRY Bin wivent, he sper in.

Che lialtely yought swor ettervand bithes,
Nor and hation he now
OLUS:
Firan leas deadvager ticer sets up sublorell beeponew o's asue
wileart chan:
Sill do thy the farry re and hate

Not so great.. but what if we increase the order to 4?

### order 4

In [11]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm, 4))

First not, the be devously tristing to see
him scape,
when
embrace to seen ther.

SOMERSET:
Fare though unfold;
My same,
these and grace, my grund thee,
When she he wear at thou,
There hearthly put
My father spoil: heart of back offician known hopes; to that's, ere hero is the naked upsider are to-night 'tis a son:
And advance: if heral, alter'd and hath smathersity.

FALSTAFF:
Truly, with and low thou must is have not with mass, halt the pale.

LAERTES:
Decoctus.
How her thin claiments; 'tis most never devotion we shall I straitors legious suitor a greath me.
Now, cheerful press' from my father. What's courtesy
Thou be fie, and brief; often my soul
The muster'd name;
If it to fear me tongue will you not few will.

BIRON:
All's banner reason. Haply my peace!

LONGAVILLE:
I shallow, and,
That were like of othereof his battle devils of this flesh powers, as I; 'tis not like the a more valouds,
I wilt some upon this, which heart.

TROILUS:
Fly not needs hadst thus.

COUNTESS OF SYRACUSE:


In [12]:
lm = train_char_lm("shakespeare_input.txt", order=4)
print(generate_text(lm, 4))

First beat me sparing eye
Of old, rides in this is our proclaim as his parce his wooing I'ld remerchief
Is all finess.

DUCHESS FORD:
Well,
By signess,
To trod of though now! are of France: you hither what permioner thy grace?

EDWARD:
What you forgot?

AENEAS:
What alongst the asleep peace, braid the throwns; what defeature was line
The false days,
either the prime anger.

CHARD III:
A father
now now silver moody counseeing become approviden make to figs.
Unless he no offendering but ta? do deputy by heart? She less exhales. Coming every time and tend mightness,
That thus do no other villain, she was lying
Cuts of the crown inful type of my night,
And e'er and
almost of it who hast,
Were eye heart;
Becomes to bade his son,--here?

LUCIUS:
Opens have, this mortime me; and fate you.

BEATRICE:
It stay lie
On embosses, from his sound and elegiant he would not be whose. I have leasier ladies his is at utter'd,
One cold. O love's deer?

DEMETRIUS:
'Tis voices have here
a man's lodge in it 

This is already quite reasonable, and reads like English. Just 4 letters history! What if we increase it to 7?

### order 7

In [13]:
lm = train_char_lm("shakespeare_input.txt", order=7)
print(generate_text(lm, 7))

First Citizen:
O noble nature native English Henry Bolingbroke
Into some paper.

EDMUND:
If it come to shame.

PROTEUS:
O, throw away twelvemonth's end
I'll peach for you,
Though I know you love me this words
He scatters
Point again,
He came early welcome: to Aufidius, welcome home?' quoth he. 'You do you needest him take't of leaping in the tranquil mind! Shall we respect
For what? let's hie!

BASTARD:
She's a verse the last to his come to us,
The people were my horses,
Where some reason is, she uses of his play
is the next for me?

PROTEUS:
You are too long;
But pay me the gods had mustachio purple with iteration of this fortune's womb,
That judgment her: welcome from the theatre, whereof is always of villany should love these ears of prison, in any thing?

KING JOHN:
We from their fortune him against sworn true noblest Rome,
Or bitterest.

GADSHILL:
As well again else. Met'st thou farthest by his better guest; so he will not be seen him that your names?
Arm, arm, my lord.

AGAMEMNON

### How about 10?

In [14]:
lm = train_char_lm("shakespeare_input.txt", order=10)
print(generate_text(lm, 10))

First Citizen:
Marry, this is thine and their merits and our crimes would prevail
Under the cool and tempest of my eyes.

THESEUS:
What are you here, sir, and welcome! I have a pilot's thumb,
Wreck'd as homeward he did contend
Which is a comforter.

ANTONIO:
Alas the heavens,
Cut short all intermingle every thing I have done well in his perfect woman, she is
indeed more than can the substance into that fair prayer
To soften Angelo: and that's myself,
The gracious.

ARCHBISHOP OF YORK:
I had a wound her honourably.
So call thee Caesar now,
No humble suppliant may
But beg one favour and
for a week; for the goose came out of love in't.

ORLANDO:
Nor shall it thee,
So long attended with the Duke of Burgundy!

BURGUNDY:
What wilt thou help'st me not?
By heaven, so she could speak with me; Lavinia, let me kiss my sovereignly being honour into lust, to take this poor mad soul; and she is
painted also with a white beard? a decreasing leg? an
increasing vine!

BELARIUS:
What's the business put 

### This works pretty well

With an order of 4, we already get quite reasonable results. Increasing the order to 7 (~word and a half of history) or 10 (~two short words of history) already gets us quite passable Shakepearan text. I'd say it is on par with the examples in Andrej's post. And how simple and un-mystical the model is!

### So why am I impressed with the neural networks after all?

Generating English a character at a time -- not so impressive in my view. The NN needs to learn the previous $n$ letters, for a rather small $n$, and that's it. 

However, the code-generation example is very impressive. Why? because of the context awareness. Note that in all of the posted examples, the code is well indented, the braces and brackets are correctly nested, and even the comments start and end correctly. This is not something that can be achieved by simply looking at the previous $n$ letters. 

If the examples are not cherry-picked, and the output is generally that nice, then the NN did learn something not trivial at all.

Just for the fun of it, let's see what our simple language model does with the linux-kernel code:

In [15]:
!wget http://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt

--2024-10-02 08:57:25--  http://cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt [following]
--2024-10-02 08:57:27--  https://www.cs.stanford.edu/people/karpathy/char-rnn/linux_input.txt
Resolving www.cs.stanford.edu (www.cs.stanford.edu)... 23.216.154.123, 23.216.154.115
Connecting to www.cs.stanford.edu (www.cs.stanford.edu)|23.216.154.123|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-10-02 08:57:28 ERROR 404: Not Found.



In [16]:
lm = train_char_lm("linux_input.txt", order=10)
print(generate_text(lm, 10))

/*
 * linux/kernel.h>
#include <linux/slab.h>
#include <linux/sort.h>

#include <linux/irq_work.h>
#include <linux/sysctl.h>
#include <linux/threads.h>
#include <linux/atomic.h>
#include <linux/string.h>
#include <linux/cred.h>
#include <linux/init.h>
#include <linux/seq_file.h>
#include <linux/interrupt.. Legacy)
	 */
	if (!param)
		goto out;
		}
	}

	ret = __trace_graph_ent,	graph_ent			= *trace;
	int ret;

	mutex_unlock(&tu->filter.nr_systemwide--;
		done = tu->filter.nr_systemwide)
		return 0;
}

/**
 * ptrace_stop(void *data;
	int cpu = (long)hcpu;

	switch (key->both.offset |= FUT_OFF_MMSHARED:
		futex_get_mm(union futex_keys that hash to the same jiffy.
 */
static void perf_event_header	header;

		u32				pid;
		u32				tid;
};

static void register state */
		if (dattr) {
			*dattr = SD_ATTR_INIT;
			update_group_times(event);
		rt_mutex_unlock(&kprobe_mutex);

	/*
	 * rb->aux_mmap_count))
		goto out_free;
	}
	current->flags & FAULT_FLAG_WRITE);
	up_read(&uts_sem);
	u = utsname()

In [17]:
lm = train_char_lm("linux_input.txt", order=15)
print(generate_text(lm, 15))

/*
 * linux/kernel/time/tick-broadcast-hrtimer.c
 * This file emulates a local clock event devices which handle everything in page size chunks ensure
	 * the destination addresses
	 * through very weird things can happen
	 * if the module is unloaded, and then by giving the lock_torture_stats_print();
		torture_shutdown task started");
	do {
		schedule_timeout_interruptible(1);
	}
	smp_mb(); /* matches sched_clock_init() */

	if (!sched_clock_irqtime) {
		irqtime_account_hi_update(void)
{
	ktime_t period;

	write_seqlock(&tsk->vtime_seqlock);
}

void * __weak arch_kexec_kernel_verify_sig(image, image->kernel_buf_len);

	if (ret < 0)
		errors++;

	num_tests++;
	ret = test_kprobe();
	if (ret < 0)
		goto out_balanced;

	if (env->idle != CPU_NOT_IDLE) &&
	    likely(p->policy != SCHED_NORMAL && policy != SCHED_DEADLINE task.
 *
 * Only the static values are converted to jiffies when they are still
	 * active. Clear the pending bitmasks, but must still be cleared in our caller bc CLONE_THRE

In [18]:
lm = train_char_lm("linux_input.txt", order=20)
print(generate_text(lm, 20))

/*
 * linux/kernel/irq/chip.c
 *
 * Copyright 2003-2007 Red Hat Inc., Durham, North Carolina.
 * Copyright 2005 Hewlett-Packard Development Company, L.P.
 * Copyright (C) 2004-2006 Tom Rini <trini@kernel.crashing.org>
 * Copyright (C) 2012 Red Hat, Inc. All Rights Reserved.
 * Copyright (c) 2003 Patrick Mochel
 * Copyright (c) 2009 Wind River Systems, Inc.  All Rights Reserved.
 * Written by David Howells (dhowells@redhat.com)
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU General Public License
 *  along with this program; if not, you can access it online at
 * http://www.gnu.org/licenses/gpl-2.0.html.
 *
 * Copyright (c) 2001   David Howells (dhowells@redhat.com).
 * - Derived partially from idea by Andrea Arcangeli <andrea@suse.de>
 * - Derived also from comments by Linus
 */
#include <linux/swap.h>
#include <linux/cpu.h>
#i

In [19]:
print(generate_text(lm, 20))

/*
 * linux/kernel/irq/autoprobe.c
 *
 * Copyright (C) IBM Corporation, 2014
 *
 * Author: Paul E. McKenney <paulmck@linux.vnet.ibm.com>
 */

#include <linux/module.h>	/* for MODULE_NAME_LEN via KSYM_SYMBOL_LEN */
#include <linux/clocksource.h>
#include <linux/slab.h>
#include <linux/mm.h>
#include <linux/percpu.h>
#include <linux/mount.h>
#include <linux/slab.h>
#include <linux/kdb.h>
#include <linux/module.h>

#include <asm/uaccess.h>

/*
 * mutex protecting text section modification (dynamic code patching).
 * some users need to sleep (allocating memory...) while they hold this lock.
 *
 * NOT exported to modules - patching kernel text is a really delicate matter.
 */
DEFINE_MUTEX(trace_types_lock);

	return ret;
}

static int update_cpumask(struct cpuset *cs, nodemask_t *new_mems,
		     bool cpus_updated, bool mems_updated)
{
	bool is_empty;

	spin_lock_irq(&callback_lock);
		if (!on_dfl)
			cpumask_copy(top_cpuset.effective_mems = node_states[N_MEMORY].
 * Call this routine anyti

In [20]:
print(generate_text(lm, 20, nletters=5000))

/*
 * linux/kernel/irq/autoprobe.c
 *
 * Copyright (C) 1992, 1998-2006 Linus Torvalds, Ingo Molnar
 * Copyright(C) 2005-2007, Red Hat, Inc., Ingo Molnar <mingo@redhat.com>
 *  Copyright (C) 2004, 2005, 2006 Red Hat, Inc., Ingo Molnar <mingo@redhat.com>
 *  Copyright (C) 2004 Pavel Machek <pavel@ucw.cz>
 * Copyright (C) 2012 Dario Faggioli <raistlin@linux.it>,
 *                                       /* 5 bit base 2 exponent, 20 bits mantissa.
 * The leading bit of the mantissa is not stored, but implied for
 * non-zero exponents.
 * Largest encodable value is 50 bits.
 */

#define MANTSIZE2       20                          |
 *      |                                         ----\n");
	return 0;
}

static int trace_selftest_startup_function_graph(struct tracer *trace, struct trace_array *tr)
{
	start_branch_trace(tr);
	return 0;
}

static int tk_debug_show_sleep_time, NULL);
}

static const struct file_operations proc_cgroupstats_operations);
	return 0;
}
device_initcall(audit_watch_in

Order 10 is pretty much junk. In order 15 things sort-of make sense, but we jump abruptly between the 
and by order 20 we are doing quite nicely -- but are far from keeping good indentation and brackets. 

How could we? we do not have the memory, and these things are not modeled at all. While we could quite easily enrich our model to support also keeping track of brackets and indentation (by adding information such as "have I seen ( but not )" to the conditioning history), this requires extra work, non-trivial human reasoning, and will make the model significantly more complex. 

Neural networks, on the other hand, seemed to have just learn it on its own. And that's impressive.

## Names dataset

Let's do a head to head comparison between n-gram LMs and a neural network that uses trigram word embeddings as input to predict the next token. 

Notice that the model is very similar to the word2vec continuous bag of words (CBOW) model except that instead of a masked token surrounded by the tokens used to predict the masked token, instead we predict the next token (as if it was masked) using the left to right context. Both models use n-gram word embeddings but n-gram LMs use a multi-layer model instead of a single layer like word2vec.

In [1]:
lm = train_char_lm("names.txt", order=4)
print(generate_text(lm, 4))

NameError: name 'train_char_lm' is not defined

## The End

In [21]:
from IPython.core.display import HTML

def css_styling():
    styles = open("../css/notebook.css", "r").read()
    return HTML(styles)
css_styling()